In [1]:
#We need these in this file:
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import numpy as np # linear algebra
import get_lodging_scores
import SVM_classifier_general as svm_general
import pickle
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
import glob

import sys
sys.path.append('cropping')
import load_read_name_extractor as lrne
from numba import jit, cuda


In [2]:
data_paths_auto = glob.glob("Features/*npy")
data_paths_auto

['Features/hog_features_(96, 192, 3)_cells_(8, 8)_block_(4, 4)_norm_L2_mir_False.npy',
 'Features/hog_features_(64, 128, 3)_cells_(4, 4)_block_(6, 6)_norm_L2_mir_False.npy',
 'Features/hog_features_(64, 128, 3)_cells_(6, 6)_block_(8, 8)_norm_L2_mir_False.npy',
 'Features/hog_features_(64, 128, 3)_cells_(8, 8)_block_(2, 2)_norm_L2_mir_False.npy',
 'Features/hog_features_(64, 128, 3)_cells_(4, 4)_block_(8, 8)_norm_L2_mir_False.npy',
 'Features/hog_features_(64, 128, 3)_cells_(6, 6)_block_(6, 6)_norm_L2_mir_False.npy',
 'Features/hog_features_(64, 128, 3)_cells_(8, 8)_block_(6, 6)_norm_L2_mir_False.npy',
 'Features/hog_features_(96, 192, 3)_cells_(4, 4)_block_(4, 4)_norm_L2_mir_False.npy',
 'Features/hog_features_(64, 128, 3)_cells_(6, 6)_block_(2, 2)_norm_L2_mir_False.npy',
 'Features/hog_features_(64, 128, 3)_cells_(8, 8)_block_(8, 8)_norm_L2_mir_False.npy',
 'Features/hog_features_(64, 128, 3)_cells_(4, 4)_block_(2, 2)_norm_L2_mir_False.npy',
 'Features/hog_features_(96, 192, 3)_cells_

In [11]:
features_paths = data_paths_auto


In [8]:
temp_path = []
for i in range(len(features_paths)):
    temp_path.append(features_paths[i].replace("Features/img_names","names/img_names"))
names_paths = temp_path
temp_path = []

In [9]:
names_paths 

['Features/img_names_(96, 192, 3)_cells_(8, 8)_block_(1, 1)_norm_L2_mir_False.npy',
 'Features/img_names_(96, 192, 3)_cells_(8, 8)_block_(2, 2)_norm_L2_mir_False.npy',
 'Features/img_names_(96, 192, 3)_cells_(8, 8)_block_(5, 5)_norm_L2_mir_False.npy',
 'Features/img_names_(96, 192, 3)_cells_(8, 8)_block_(6, 6)_norm_L2_mir_False.npy',
 'Features/img_names_(96, 192, 3)_cells_(8, 8)_block_(7, 7)_norm_L2_mir_False.npy',
 'Features/img_names_(96, 192, 3)_cells_(10, 10)_block_(4, 4)_norm_L2_mir_False.npy',
 'Features/img_names_(96, 192, 3)_cells_(12, 12)_block_(4, 4)_norm_L2_mir_False.npy',
 'Features/img_names_(96, 192, 3)_cells_(8, 8)_block_(3, 3)_norm_L2_mir_False.npy',
 'Features/img_names_(96, 192, 3)_cells_(8, 8)_block_(4, 4)_norm_L2_mir_False.npy',
 'Features/img_names_(72, 248, 3)_cells_(8, 8)_block_(1, 1)_norm_L2_mir_False.npy',
 'Features/img_names_(64, 128, 3)_cells_(8, 8)_block_(3, 3)_norm_L2_mir_False.npy',
 'Features/img_names_(72, 248, 3)_cells_(10, 10)_block_(4, 4)_norm_L2_mi

In [10]:
#features_load = np.load(features_paths[0], allow_pickle=True)
#print(features_load.shape)
#print(features_load[0])
#features_paths[0]

In [11]:
#np.load(names_paths[0], allow_pickle=True) == np.load(names_paths[1], allow_pickle=True)

In [12]:
#features_load[3000].shape

In [13]:
def run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf", class_weights = None, date_ = False):
    for i in range (len(features_paths)):
                #laod features and names
        features_load = np.load(features_paths[0], allow_pickle=True)
        names_load = np.load(names_paths[0], allow_pickle=True)
        
        features_load = features_load.tolist()

        #load labels
        bin_size = 10
        labels = svm_general.load_labels(bin_size)

        #convert names into [flight_folder, ROI]
        names = lrne.it_name_extract_labels_from_img_jpeg(names_load)
        names_load = None
        
        #match labels with feature names
        features_load = svm_general.match_pic_label_to_names_new(features_load, labels, names, date_)
        data_frame = features_load
        features_load = None
        labels = None
        names = None
        
        #shuffle data and convert to array
        np.random.shuffle(data_frame)

        #partition data
        percentage = 80
        #data_frame = np.array(data_frame)
        x_train, x_test, y_train, y_test = svm_general.partition_data_list(data_frame, percentage)
        data_frame = None
        features_aar = None

        #create svm
        clf = svm.SVC(kernel=svm_kernel, class_weight=class_weights, verbose=True)
        #fit model with training data
        clf.fit(x_train,y_train)
        print("Fit Completed")
        #predict data
        y_pred = clf.predict(x_test)
        print("Predict Completed")

        #Save the found model
        n_bins = int(np.floor(100/bin_size))
        path = "SVMC_" + "Kernel_" + svm_kernel + "_Nbins_" + str(n_bins) + "_ClassWeights_" + str(class_weights) + "_date" + str(date_) + "_" + features_paths[i].replace("Features/", "")
        
        #Save the results
        results = "Accuracy: "+str(accuracy_score(y_test, y_pred)) + "\n" + classification_report(y_test, y_pred)
        svm_general.save_results(results, path)
        #Save the found model
        svm_general.save_model(clf, path)



In [ ]:
run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf", class_weights="balanced", date_=True)

1
2
3
Data Partitioned Successfully
[LibSVM]..
*
optimization finished, #iter = 2404
obj = -651.900448, rho = 0.155295
nSV = 2349, nBSV = 1982
*.*
optimization finished, #iter = 1139
obj = -466.884655, rho = 0.733354
nSV = 1010, nBSV = 647
*.*
optimization finished, #iter = 1106
obj = -411.166065, rho = 1.911225
nSV = 952, nBSV = 524
*.*
optimization finished, #iter = 1137
obj = -336.759081, rho = 1.833826
nSV = 880, nBSV = 382
*.*
optimization finished, #iter = 986
obj = -241.851895, rho = 1.856610
nSV = 722, nBSV = 280
*
optimization finished, #iter = 986
obj = -258.809036, rho = 2.424693
nSV = 781, nBSV = 288
*
optimization finished, #iter = 794
obj = -159.551997, rho = 1.575842
nSV = 560, nBSV = 133
*
optimization finished, #iter = 194
obj = -6.678580, rho = -0.791837
nSV = 141, nBSV = 0
.
*
optimization finished, #iter = 1866
obj = -483.211998, rho = 0.818640
nSV = 1794, nBSV = 1341
.
*
optimization finished, #iter = 1830
obj = -428.602107, rho = 1.812982
nSV = 1692, nBSV = 1170
.

/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model Saved Succesfull
1
2
3
Data Partitioned Successfully
[LibSVM]..
*
optimization finished, #iter = 2404
obj = -656.816940, rho = 0.327781
nSV = 2355, nBSV = 2001
.*
optimization finished, #iter = 1178
obj = -476.438120, rho = 0.840780
nSV = 1024, nBSV = 662
*.*
optimization finished, #iter = 1132
obj = -402.459618, rho = 1.836545
nSV = 939, nBSV = 512
*.*
optimization finished, #iter = 1178
obj = -339.558947, rho = 1.983817
nSV = 884, nBSV = 383
*.*
optimization finished, #iter = 979
obj = -232.512329, rho = 1.790094
nSV = 703, nBSV = 261
*
optimization finished, #iter = 976
obj = -256.612001, rho = 2.465517
nSV = 769, nBSV = 271
*
optimization finished, #iter = 780
obj = -167.015416, rho = 1.726039
nSV = 582, nBSV = 156
*
optimization finished, #iter = 186
obj = -6.854989, rho = -0.810526
nSV = 135, nBSV = 0
.
*
optimization finished, #iter = 1901
obj = -493.263174, rho = 0.824306
nSV = 1820, nBSV = 1380
.
*
optimization finished, #iter = 1833
obj = -427.327681, rho = 1.715078
nSV

/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model Saved Succesfull
1
2
3
Data Partitioned Successfully
[LibSVM]..
*
optimization finished, #iter = 2432
obj = -652.399973, rho = 0.335715
nSV = 2377, nBSV = 1995
.*
optimization finished, #iter = 1167
obj = -462.122857, rho = 0.739315
nSV = 994, nBSV = 628
*.*
optimization finished, #iter = 1111
obj = -394.588582, rho = 1.813007
nSV = 928, nBSV = 481
*.*
optimization finished, #iter = 1142
obj = -330.655435, rho = 1.957193
nSV = 872, nBSV = 372
*.*
optimization finished, #iter = 1013
obj = -235.586536, rho = 1.957832
nSV = 715, nBSV = 259
*
optimization finished, #iter = 973
obj = -256.619722, rho = 2.427716
nSV = 764, nBSV = 278
*
optimization finished, #iter = 813
obj = -168.346121, rho = 1.658470
nSV = 579, nBSV = 154
*
optimization finished, #iter = 195
obj = -4.576208, rho = -0.856402
nSV = 153, nBSV = 0
.
*
optimization finished, #iter = 1909
obj = -482.477059, rho = 0.728763
nSV = 1829, nBSV = 1394
.
*
optimization finished, #iter = 1832
obj = -422.181426, rho = 1.683251
nSV

...

Model Saved Succesfull
1
2
3
Data Partitioned Successfully
[LibSVM]..
*
optimization finished, #iter = 2377
obj = -654.519496, rho = 0.292781
nSV = 2330, nBSV = 2005
.*
optimization finished, #iter = 1179
obj = -472.690258, rho = 0.809316
nSV = 1008, nBSV = 645
*.*
optimization finished, #iter = 1121
obj = -411.664450, rho = 2.020133
nSV = 939, nBSV = 524
*.*
optimization finished, #iter = 1124
obj = -323.640967, rho = 1.882964
nSV = 854, nBSV = 346
*.*
optimization finished, #iter = 1003
obj = -241.844343, rho = 1.939340
nSV = 712, nBSV = 269
*
optimization finished, #iter = 971
obj = -254.015135, rho = 2.477391
nSV = 764, nBSV = 268
*
optimization finished, #iter = 778
obj = -167.045069, rho = 1.703800
nSV = 566, nBSV = 146
*
optimization finished, #iter = 161
obj = -4.687007, rho = -0.863469
nSV = 124, nBSV = 0
.
*
optimization finished, #iter = 1877
obj = -495.148811, rho = 0.732607
nSV = 1811, nBSV = 1390
.
*
optimization finished, #iter = 1862
obj = -439.587160, rho = 1.806212
nS

/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model Saved Succesfull
1
2
3
Data Partitioned Successfully
[LibSVM]..
*
optimization finished, #iter = 2407
obj = -650.723193, rho = 0.157642
nSV = 2363, nBSV = 1983
*.*
optimization finished, #iter = 1142
obj = -458.964845, rho = 0.688937
nSV = 988, nBSV = 606
*.*
optimization finished, #iter = 1113
obj = -397.957876, rho = 1.821886
nSV = 921, nBSV = 498
*.*
optimization finished, #iter = 1134
obj = -344.425923, rho = 1.831623
nSV = 868, nBSV = 395
*.*
optimization finished, #iter = 998
obj = -250.852815, rho = 1.906870
nSV = 732, nBSV = 279
*
optimization finished, #iter = 979
obj = -260.524766, rho = 2.409712
nSV = 771, nBSV = 285
*
optimization finished, #iter = 800
obj = -168.338352, rho = 1.675156
nSV = 571, nBSV = 150
*
optimization finished, #iter = 157
obj = -4.496125, rho = -0.839141
nSV = 127, nBSV = 0
.
*
optimization finished, #iter = 1900
obj = -474.914581, rho = 0.761354
nSV = 1802, nBSV = 1333
.
*
optimization finished, #iter = 1838
obj = -420.208709, rho = 1.766069
nSV

/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model Saved Succesfull
1
2
3
Data Partitioned Successfully
[LibSVM]..
*
optimization finished, #iter = 2426
obj = -653.532407, rho = 0.272625
nSV = 2359, nBSV = 1997
.*
optimization finished, #iter = 1149
obj = -476.828083, rho = 0.861582
nSV = 1030, nBSV = 689
*.*
optimization finished, #iter = 1155
obj = -406.698474, rho = 1.970240
nSV = 956, nBSV = 514
*.*
optimization finished, #iter = 1172
obj = -330.756232, rho = 1.984652
nSV = 887, nBSV = 378
*.*
optimization finished, #iter = 1014
obj = -244.979334, rho = 1.927345
nSV = 750, nBSV = 278
*
optimization finished, #iter = 977
obj = -253.833802, rho = 2.408231
nSV = 771, nBSV = 283
*
optimization finished, #iter = 760
obj = -164.906038, rho = 1.614285
nSV = 570, nBSV = 156
*
optimization finished, #iter = 218
obj = -8.748000, rho = -0.734812
nSV = 140, nBSV = 0
.
*
optimization finished, #iter = 1908
obj = -496.564863, rho = 0.886415
nSV = 1835, nBSV = 1416
.
*
optimization finished, #iter = 1838
obj = -427.713072, rho = 1.814940
nS

In [ ]:
print(features_paths[0])
print(len(features_paths))1148